# **사용자 정의값**
### 입력 후 아래의 코드를 실행해주세요

**[인증키 설정]**
- hb_api_key : 한국은행 오픈API 인증키
- drt_api_key : DART 오픈API 인증키

**[산업 검색값 정의]**
- lang : 결과값 언어 ('kr', 'en')
- industry_cd : 업종코드 (식료품:C10)
- corp_size : 기업규모 (종합:A, 대기업:L, 대기업_중견:J, 중소기업:M)
- period : 주기 (년:A, 반년:S, 분기:Q, 월:M, 반월:SM, 일: D)
- search_start : 검색시작일자, 주기에 맞는 형식으로 입력: 2015, 2015Q1, 201501, 20150101 등
- search_end : 검색종료일자, 위와 동일

**[기업 검색값 정의]**
- corp : 기업 이름 입력 !오탈자 주의
- reprt_code : 보고서 종류 (1분기보고서:11013, 반기보고서:11012, 3분기보고서:11014, 사업보고서:11011)
- fs_div : 재무제표 종류 (OFS:재무제표, CFS:연결재무제표)

In [1]:
# 인증키 설정
hb_api_key = 'your_key' 
drt_api_key = 'your_key' 

# 산업 검색값
lang = 'kr' 
industry_cd = 'C10'
corp_size = 'A'
period = 'A'
search_start = 2016
search_end = 2018

# 기업 검색값
corp = '동원F&B'
bsns_year = 2018,
reprt_code = 11011 
fs_div = 'OFS' 



---



### 라이브러리 정의

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from io import BytesIO
import zipfile
import xml.etree.ElementTree as ET

import warnings
warnings.filterwarnings('ignore')

### 클래스 정의
- ***IndustryStats*** : 한국은행 기업분석통계자료 추출 클래스
        - 필요 인자 -> api_key, lang, industry_cd, corp_size, period, search_start, search_end
    - ***get_openAPI*** : get_stats의 하위함수, 제무상태표와 같이 구성비가 필요한 자료를 df로 추출하는 함수
        - 필요 인자 -> stats_cd : 각 통계코드
    - ***get_dfSum*** : get_stats의 하위함수, 제무상태표와 같이 구성비가 필요한 자료를 df로 추출하는 함수
        - 필요 인자 -> stats_cd : 각 통계코드
    - ***get_dfUnit*** : get_stats의 하위함수, 이익잉여금처분계산서와 같이 3개년 비율이 필요한 자료를 df로 추출하는 함수
        - 필요 인자 -> stats_cd : 각 통계코드, search_end : 검색종료 일자
    - ***get_stats*** : 기업경영분석 통계자료 수집하는 함수
    - 필요 인자 -> lang, industry_cd, corp_size, period, search_start, search_end, indicators_cd
- ***CorpCode*** : DART 기업고유번호 추출 클래스
    - ***get_dfCorpCode*** : 기업고유번호 확인하는 함수
        - 필요 인자 -> api_key : 인증키
- ***FinancialStatements*** : DART 기업 전체 재무제표 추출 클래스
    - ***get_tag*** : 보고서 xml로 추출하는 함수
        - 필요 인자 -> api_key : 인증키, corp_code : 기업고유번호
    - ***get_df*** : xml -> df로 변환하는 함수
        - 필요 인자 -> fs_name : 재무제표 이름
    - ***get_stats*** : 전체 재무제표 df를 담은 딕셔너리
        - key : 재무제표 이름, value : 재무제표 df

In [3]:
class IndustryStats:
    def __init__(self):
        self.api_key = hb_api_key
        self.lang = lang
        self.period = period
        self.search_start = search_start
        self.search_end = search_end
        self.industry_cd = industry_cd
        self.corp_size = corp_size
        self.indicators_cd = {'501Y001':'재무상태표','501Y002':'손익계산서','501Y003':'제조원가명세서','501Y004':'이익잉여금처분계산서',
                         '501Y005':'성장성지표','501Y006':'손익관계비율','501Y007':'자산/자본비율','501Y008':'자산/자본회전율',
                         '501Y009':'생산성지표','501Y010':'부가가치구성'}
    
    def get_openAPI(self, stats_cd):
        # openAPI 파서
        url = 'https://ecos.bok.or.kr/api/StatisticSearch/{key}/xml/{lang}/1/1000/{code}/{period}/{start}/{end}/{indst}/{crpr}'
        api_url = url.format(key=self.api_key, lang=self.lang, 
                             code=stats_cd, period=self.period, start=self.search_start, end=self.search_end, 
                             indst=self.industry_cd,crpr=self.corp_size)
        return BeautifulSoup(requests.get(api_url).text, 'lxml')

    def get_dfSum(self, rs_xml):
        # 데이터 추출
        data_set = []
        unit = rs_xml.find_all('unit_name')[0].text.strip()
        for i in range(len(rs_xml.find_all('row'))):
            data_set.append((rs_xml.find_all('time')[i].text, rs_xml.find_all('item_code3')[i].text, 
                             rs_xml.find_all('item_name3')[i].text, rs_xml.find_all('data_value')[i].text))
        df = pd.DataFrame(data_set, columns=['연도', 'code_no', '내역', f'금액({unit})'])
        df[f'금액({unit})'] = df[f'금액({unit})'].astype(float)
        df['구성비(%)'] = 0
        for year in df['연도'].unique():
            year_max = df[df['연도']==year][f'금액({unit})'].max()
            idx = df[df['연도']==year].index.to_list()
            df.iloc[idx, 4] = ((df.iloc[idx, 3] / year_max)*100).round(2) 
        df[f'금액({unit})'] = df[f'금액({unit})'].apply(lambda x: '{:.0f}'.format(x))
        return df
    
    def get_dfUnit(self, rs_xml, search_end):
        # 데이터 추출
        data_set = []
        for i in range(len(rs_xml.find_all('row'))):
            data_set.append((rs_xml.find_all('item_code3')[i].text, rs_xml.find_all('item_name3')[i].text,
                             rs_xml.find_all('time')[i].text, rs_xml.find_all('data_value')[i].text, 
                             rs_xml.find_all('unit_name')[i].text))
        temp = pd.DataFrame(data_set, columns=['code_no','내역','year','value','단위'])
        df = temp.pivot(index=['code_no','내역'], columns='year', values='value').reset_index()
        df.rename_axis(index=None, columns=None, inplace=True)
        df['단위'] = temp['단위'].unique()[0]
        return df

    def get_stats(self):
        # 통계자료 수집
        stats = {}
        for stats_cd, stats_name in self.indicators_cd.items():
            # 금액 자료용
            if stats_cd in ['501Y001','501Y002','501Y003']:
                stats[stats_name] = self.get_dfSum(self.get_openAPI(stats_cd))
            # 비율 자료용
            else:
                # 단 건 조회 시, 비교를 위해 3년 전 데이터도 함께 추출
                if self.search_start == self.search_end:
                    self.search_end += 2
                stats[stats_name] = self.get_dfUnit(self.get_openAPI(stats_cd), self.search_end)
        return stats
# ----------------------------------------------------------------------------------------------- #
class CorpCode:
    def __init__(self):       
        def get_df(self, api_key):
            # OpenAPI 데이터 불러오기
            url = '	https://opendart.fss.or.kr/api/corpCode.xml'
            params = {'crtfc_key':api_key}
            response = requests.get(url, params=params)
            file = BytesIO(response.content) # 바이너리 스트림형태로 메모리에 저장
            zfile = zipfile.ZipFile(file, 'r') # zipfile 읽기
            
            # OpenAPI 데이터 파싱
            with zfile.open('CORPCODE.xml') as xfile: # zip파일 안 xml파일 열기
                tree = ET.parse(xfile)
                root = tree.getroot()
            
            tags_list = root.findall('list')
            tags_list_dict = [{child.tag: child.text for child in tag}\
                              for tag in tags_list]
            df = pd.DataFrame(tags_list_dict)
            return df

        self.api_key = drt_api_key
        self.df = get_df(self, self.api_key)

    def get_cd(self, search_corp):
        try:
            return self.df[self.df['corp_name']==f'{search_corp}'][['corp_code', 'corp_name']]\
                .to_dict(orient='records')[0]
        except IndexError:
            return print('조회된 기업이 없습니다.')
# ----------------------------------------------------------------------------------------------- #
class FinancialStatements:
    def __init__(self):
        self.api_key = drt_api_key
        self.corp_code = CorpCode().get_cd(corp)['corp_code']
        self.bsns_year = bsns_year
        self.reprt_code = reprt_code
        self.fs_div = fs_div
                
        def get_tag(self):
            # OpenAPI 데이터 불러오기
            url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.xml'
            params = {'crtfc_key':self.api_key,
                      'corp_code':self.corp_code,
                      'bsns_year':self.bsns_year,
                      'reprt_code':self.reprt_code, #1분기보고서:11013, 반기보고서:11012, 3분기보고서:11014, 사업보고서:11011
                      'fs_div':self.fs_div} # OFS:재무제표, CFS:연결재무제표
            return BeautifulSoup(requests.get(url, params=params).text, 'lxml')
            
        self.rs_xml = get_tag(self)
        self.tag_dict ={# 'rcept_no':'접수번호',
                        # 'reprt_code':'보고서 코드',
                        # 'bsns_year':'사업 연도',
                        # 'corp_code':'고유번호',
                        # 'sj_div':'재무제표구분',
                        # 'sj_nm':'재무제표명',
                        # 'account_id':'계정ID',
                        'account_nm':'계정명',
                        'account_detail':'계정상세',
                        'thstrm_nm':'당기명',
                        'thstrm_amount':'당기금액',
                        'thstrm_add_amount':'당기누적금액',
                        'frmtrm_nm':'전기명',
                        'frmtrm_amount':'전기금액',
                        'frmtrm_q_nm':'전기명(분/반기)',
                        'frmtrm_q_amount':'전기금액(분/반기)',
                        'frmtrm_add_amount':'전기누적금액',
                        'bfefrmtrm_nm':'전전기명',
                        'bfefrmtrm_amount':'전전기금액',
                        'ord':'계정과목 정렬순서',
                        'currency':'통화 단위'}

    def get_df(self, fs_name):
        try:
            values = []
            for lst in self.rs_xml.findAll('list'):
                if lst.find('sj_nm').text == fs_name:      
                    v = []
                    for tag in self.tag_dict:
                        try: value = lst.find(tag).text
                        except: value = ''
                        v.append(value)
                    values.append(v)
            
            temp = pd.DataFrame(values, columns=self.tag_dict.values())
            df = temp[['계정명', '당기금액', '전기금액', '전전기금액']]
            return df.rename(columns={'당기금액':temp['당기명'].unique()[0] + f" ({temp['통화 단위'].unique()[0]})",
                                    '전기금액':temp['전기명'].unique()[0] + f" ({temp['통화 단위'].unique()[0]})",
                                    '전전기금액':temp['전전기명'].unique()[0] + f" ({temp['통화 단위'].unique()[0]})"})
        except IndexError:
            pass
            
    def get_stats(self):
        # 통계자료 수집
        stats = {}
        for div in ['재무상태표', '손익계산서', '포괄손익계산서', '현금흐름표']:
            stats[div] = self.get_df(div)
        return stats

### 데이터 수집 및 추출

In [4]:
# 산업분석지표
idst_stats = IndustryStats().get_stats()
# 기업분석지표
corp_stats = FinancialStatements().get_stats()

In [38]:
data = [[corp, '재고자산회전율'], [corp, '총자산회전율'], ['산업평균', '재고자산회전율'], ['산업평균', '총자산회전율']]
for year in range(search_start, search_end+1):
    # 산업 재고자산
    idst_inventory = idst_stats['재무상태표'][(idst_stats['재무상태표']['연도']==f'{year}')&\
                                         (idst_stats['재무상태표']['내역']=='재고자산')]['금액(백만원)'].values[0]
    # 산업 자산총계
    idst_assets = idst_stats['재무상태표'][(idst_stats['재무상태표']['연도']==f'{year}')&\
                                             (idst_stats['재무상태표']['내역']=='자산총계')]['금액(백만원)'].values[0]
    # 산업 매출액
    idst_sales = idst_stats['손익계산서'][(idst_stats['손익계산서']['연도']==f'{year}')&\
                                          (idst_stats['손익계산서']['내역']=='매출액')]['금액(백만원)'].values[0]
    data[2].append(format(int(idst_sales)/int(idst_inventory),'.2f'))
    data[3].append(format(int(idst_sales)/int(idst_assets),'.2f'))

for i in range(search_end-search_start+1, 0, -1):
    # 기업 재고자산
    corp_inventory = int(corp_stats['재무상태표'][corp_stats['재무상태표']['계정명']=='재고자산'].iloc[:,i].values[0])
    # 기업 자산총계
    corp_assets = int(corp_stats['재무상태표'][corp_stats['재무상태표']['계정명']=='자산총계'].iloc[:,i].values[0])
    # 기업 매출액
    corp_sales = int(corp_stats['포괄손익계산서'][corp_stats['포괄손익계산서']['계정명']=='매출액'].iloc[:,i].values[0])
    data[0].append(format(int(corp_sales)/int(corp_inventory),'.2f'))
    data[1].append(format(int(corp_sales)/int(corp_assets),'.2f'))


cols = ['', '재무비율'] + list(range(search_start, search_end+1))
pd.DataFrame(data, columns=cols)

,,재무비율,2016,2017,2018
0,동원F&B,재고자산회전율,6.65,6.51,6.60
1,동원F&B,총자산회전율,1.67,1.49,1.59
2,산업평균,재고자산회전율,9.00,8.95,8.36
3,산업평균,총자산회전율,1.13,1.05,1.10
